In [58]:
# This notebook contains a computation of the weight 13 Euler characteristic of M_{g,n} for small g,n

In [59]:
# Definition of the rings of symmetric functions
# We use three version of the same ring: Sage symmetric functions in the Schur basis, in the power sum basis, 
# and plain polynomials in variables p (representing power sums)
# mind that the plain p uses 0-based indexing, i.e., p[k] stands for the power sum p_{k+1}
s = SymmetricFunctions(QQ).schur()
q = SymmetricFunctions(QQ).power()
Rv.<v> = PowerSeriesRing(q, default_prec=120)
Rw.<w> = PowerSeriesRing(s, default_prec=120)

RP = PolynomialRing(QQ, 60, names='p')
p=RP.gens()
Ru.<u> = PowerSeriesRing(RP, default_prec=120)

# conversion between the rings
def pstoqs(X):
    return sum(c * prod( q([i+1])^a for i,a in enumerate(exp) ) for exp, c in zip(X.exponents(), X.coefficients()) )
def stops(X):
    return sum(c* prod(p[i-1] for i in a) for a, c in q(X))
def pstoqsu(X):
    return O(v^(X.prec())) + sum( v^i * pstoqs(c) for i, c in zip(X.exponents(), X.coefficients()) )
def pstosu(X):
    return O(w^(X.prec())) + sum( w^i * s(pstoqs(c)) for i, c in zip(X.exponents(), X.coefficients()) )
def mulwtos(X):
    return O(w^(X.prec())) + sum( w^i * sum(s(a)*b*w^sum(a) for a,b in c) for i, c in zip(X.exponents(), X.coefficients()))
def mulvtoq(X):
    return O(v^(X.prec())) + sum( v^i * sum(q(a)*b*w^sum(a) for a,b in c) for i, c in zip(X.exponents(), X.coefficients()))
def stoqsw(X):
    return O(v^(X.prec())) + sum( v^i * q(c) for i, c in zip(X.exponents(), X.coefficients()) )

# standard functions
def PletN_s(X, n):
    cutoff = X.prec()
    accu = O(w^cutoff)
    for i, c in zip(X.exponents(), X.coefficients()):
        if n*i < cutoff:
            accu = accu + w^(n*i) * s(q([n])).plethysm(c)
    
    return accu

def SymSquare_s(X):
    return (X*X + PletN_s(X,2))*(1/2)
def PletN(X, n):
    cutoff = X.prec()
    accu = O(u^cutoff)
    for i, c in zip(X.exponents(), X.coefficients()):
        if n*i < cutoff:
            accu = accu + u^(n*i) * PletSub(c,n)
    
    return accu
def PletSub(Y, n):
    nvars = Y.parent().ngens()
    tp = Y.parent().gens()
    return Y(* [tp[n*(i+1)-1 ] if n*(i+1)<=nvars else 0 for i in range(nvars) ] )
    
def PletExp(X):
    cutoff = X.prec()
    return exp(sum(PletN(X,n)/n for n in range(1,cutoff) ) )

def PletLog(X):
    cutoff = X.prec()
    return sum( moebius(n) * log(PletN(X, n)) /n for n in range(1,cutoff) )

def SymSquare(X):
    #computes plethysm with trivial S_2 rep
    return (X*X + PletN(X,2))/2

In [97]:
# Functions
def derilist(a):
    return [p[j-1] for j in a]

def PolyDerivatives(X, Y):
    # computes X( i d/dp_i) Y, no u's in X here
    # assumes X is in power sum basis (ring "q") already
    # take derivatives
    return sum( c * (prod(a) *Y.derivative(*[p[j-1] for j in a]) if len(a)>0 else Y) for a, c in X )

def PolyDerivativesu(X,Y):
    return sum(PolyDerivatives(c,u^n * Y) for n, c in zip(X.exponents(), X.coefficients()))

def transpose_yng(X):
    # transposes all young diagrams and multiplies by -1 for odd number of boxes
    tpl = [-v for v in X.base_ring().gens()]
    return X.map_coefficients(lambda poly : poly(*tpl) )

def DisplayPoly(X, maxg=99999, maxn=99999):
    ddd = {}
    for i, c in zip(X.exponents(), X.coefficients()):
        for cc, mo in c:
            #print(i,mo,cc)
            nn = sum(cc)
            gg = i - nn
            if (gg,nn) in ddd:
                ddd[(gg,nn)] += s(cc)*mo
            else:
                ddd[(gg,nn)] = s(cc)*mo
    for (gg,nn),cc in sorted(ddd.items()):
        if gg <= maxg and nn <= maxn:
            print(color.BOLD+"g=",gg,", n=", nn, ": "+color.END, cc)

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def toLatexTable(P,maxg, maxn, maxC=-1):
    nCols = maxn
    nRows = maxg
    if maxC<0:
        maxC = P.prec()-1
    outtex = "\\begin{tabular}{|g|" + "|".join(["M" for i in range(nCols+1)]) + "|} \\hline \\rowcolor{Gray}" + \
        " g,n & " + " & ".join([str(i) for i in range(nCols+1)]) + "\\\\ \n\\hline\n"
    pd = [ [(P[g+n].truncate(n+1)-P[g+n].truncate(n) if g+n<=maxC else "?") for n in range(maxn+1)] for g in range(maxg+1) ]
    for g in range(nRows+1):
        outtex += str(g)
        for n in range(nCols+1):
            outtex += " & $" + (latex(pd[g][n]) if g+n <= maxC and len(pd)>g and len(pd[g])>n else " ") + "$"
        outtex += " \\\\  \n\\hline\n"
    outtex += "\\hline\n\\end{tabular}"
    return outtex

In [98]:
# Euler characteristics of barM_{g,n}
def Mbar2_s(maxg, maxn):
    res = O(w^(maxg+1))
    # kappa class, g>=3
    res = res + sum( w^g * s([n]) for g in range(3,maxg+1) for n in range(0,maxn+1) )
    # psi classes, g>=2
    res = res + sum( w^g * s([1]) * s([n-1]) for g in range(2,maxg+1) for n in range(1,maxn+1) )
    # delta_irr, g>=1
    res = res + sum( w^g * s([n]) for g in range(1,maxg+1) for n in range(0,maxn+1) if 2*g+n>=3 )
    ## delta classes g>=0, without relations
    # non-symmetric part
    res = res + sum(w^(g1+g2) * s([n1]) * s([n2]) for g1 in range(maxg+1) for g2 in range(g1+1) 
                                                  for n1 in range(maxn+1) for n2 in range(maxn-n1+1) 
                                                  if 2*g1+n1>=2 and 2*g2+n2>=2 and g1+g2<=maxg and not (g1==g2 and n1<=n2)) 
    # symmetric part
    res = res + sum(w^(2*g) * s([2]).plethysm(s([n]))  for g in range(maxg//2+1) 
                                                       for n in range(maxn//2+1) 
                                                       if 2*g+n>=2)
    # add relations and psis in g=0
    res = res + sum( s([1]) * s([n-1]) - s([2])*s([n-2]) for n in range(4,maxn+1))
    return res
def ComOp(minval, maxval):
    if maxval == 0 and minval == 0:
        return O(u)+1
    X = exp(O(u^(maxval+1)) + sum(u^n * p[n-1]/n for n in range(1, maxval+1)))
    return X - X.truncate(minval)

def Mbar2(cutoff):
    res = O(u^cutoff)
    # kappa class, g>=3
    res = res + sum( u^j * ComOp(0,cutoff-j-1) for j in range(3,cutoff) )
    # psi classes, g>=2
    res = res + sum( u^(j+1) * p[0] * ComOp(0,cutoff-j-2) for j in range(2,cutoff-1) )
    # delta_irr, g>=1
    res = res + u*ComOp(1,cutoff-2) + sum( u^j * ComOp(0,cutoff-j-1) for j in range(2,cutoff) )
    ## delta classes g>=1
    # without relations, 
    oneside = ComOp(2,cutoff-1) + sum( u^j * ComOp(0,cutoff-j-1) for j in range(1,cutoff) )
    res = res + SymSquare(oneside)
    # add relations .. psis in g=0
    res = res + u * p[0] * ComOp(2,cutoff-2)
    # add relations .. E_ij in g=0 with opposite sign
    res = res - u^2 * (p[0]^2 +p[1]) * ComOp(1,cutoff-3) / 2
    return res

def Mbar11_s(maxn):
    if maxn<11:
        return 0
    else:
        return -w * sum(s([n-10]+[1 for _ in range(10)]) for n in range(11,maxn+1) )

def Mbar13_s(maxg,maxn):
    res = O(w^(maxg+1))
    a10_s = s([1 for _ in range(10)])
    hook2_10_s = s([2] + [1 for _ in range(10)])
    
    # g>=2 contribution
    res = res - sum(w^g * a10_s * s([n1]) * s([n2]) for g in range(2,maxg+1)
                                                    for n1 in range(0, maxn+1-10)
                                                    for n2 in range(0, maxn-n1+1-10)
                   )
    # g=1 contribution
    res = res - w * hook2_10_s * sum(s([n-12]) for n in range(12,maxn+1))
    res = res - w*a10_s* sum(s([n1]) * s([n2]) for n1 in range(3, maxn+1-10)
                                               for n2 in range(0, maxn-n1+1-10))
    
    return res


In [62]:
## Tsopmene-Turchin formula
def B(X, cutoff):
    Y = (1/X).power_series() +O(u^cutoff)
    return sum(  Y^(k-1) * bernoulli(k) / (k*(k-1)) for k in range(2, cutoff)) +O(u^cutoff)
def El(l):
    return sum(moebius(l/d)*u^(-d) for d in divisors(l) )/l
def lal(l):
    return u^l*(1-u^l)*l
def logUl(X, l, cutoff):
    return ( X*(log( (lal(l)*El(l)).power_series() ) -1) +(-El(l)+X-1/2)*log((1-X/El(l)).power_series())).power_series()+B(-El(l)+X, cutoff)-B(-El(l), cutoff) 
def logUlp(l, cutoff):
    return logUl(sum(moebius(l/d)*p[d-1] for d in divisors(l) )/l,l,cutoff)

# this is the main routine, giving the EC for non-connected graphs
def TT_bigZ(cutoff):
    # need to go to twice the cutoff
    return exp(sum(logUlp(l,cutoff) for l in range(1, 2*cutoff) ))

In [51]:
# compute and save the Tsopmnene-Turchin Euler characteristic
%time bigZ_18 = TT_bigZ(18)
save(bigZ_18, "data/TT_bigZ_18")

CPU times: user 36.5 s, sys: 442 ms, total: 36.9 s
Wall time: 37.1 s


In [64]:
# compute and save the euler characteristics of barM_{g,n} 
%time chi2_16_32 = Mbar2_s(16,32)
save(chi2_16_32, "data/chi2_16_32")

CPU times: user 2h 13min 21s, sys: 28 s, total: 2h 13min 49s
Wall time: 2h 14min 6s


In [65]:
%time chi11_32 = Mbar11_s(32)
save(chi11_32, "data/chi11_32")

CPU times: user 515 μs, sys: 3 μs, total: 518 μs
Wall time: 520 μs


In [66]:
%time chi13_16_32 = Mbar13_s(16,32)
save(chi13_16_32, "data/chi13_16_32")

CPU times: user 826 ms, sys: 815 μs, total: 827 ms
Wall time: 827 ms


In [67]:
## compute the weight 13 euler characteristic

In [68]:
# load data
bigZ = load("data/TT_bigZ_18")

chi2_s= load("data/chi2_16_32")
chi11_s=load("data/chi11_32")
chi13_s=load("data/chi13_16_32")


In [69]:
print("Product...")
%time chi211_s=chi2_s*chi11_s

Product...
CPU times: user 1min 10s, sys: 196 ms, total: 1min 10s
Wall time: 1min 10s


In [87]:
# to power sum (q) basis
%time chi2 = stoqsw(chi2_s)
%time chi11 = stoqsw(chi11_s + O(w^50))
%time chi13 = stoqsw(chi13_s)
%time chi211 = stoqsw(chi211_s.map_coefficients(lambda X: X.truncate(32)))

CPU times: user 2min 29s, sys: 795 ms, total: 2min 30s
Wall time: 2min 30s
CPU times: user 1.2 s, sys: 1.89 ms, total: 1.2 s
Wall time: 1.2 s
CPU times: user 9min 33s, sys: 1.35 s, total: 9min 34s
Wall time: 15min 5s
CPU times: user 26min 40s, sys: 2.99 s, total: 26min 43s
Wall time: 26min 52s


In [71]:
# save data (optional)
save(chi2, "data/chi2")
save(chi11, "data/chi11")
save(chi13, "data/chi13")
save(chi211, "data/chi211")


In [72]:
%time xx_2 = PolyDerivativesu(chi2, bigZ)/bigZ

CPU times: user 2h 32min 56s, sys: 2min 55s, total: 2h 35min 52s
Wall time: 2h 36min 47s


In [73]:
%time xx_11 = PolyDerivativesu(chi11, bigZ)/bigZ

CPU times: user 5min 38s, sys: 4.82 s, total: 5min 42s
Wall time: 5min 43s


In [88]:
%time xx_all = PolyDerivativesu(chi13+chi211/v, bigZ)/bigZ

CPU times: user 1h 6min 36s, sys: 53.4 s, total: 1h 7min 29s
Wall time: 1h 7min 50s


In [89]:
# save data (optional)
save(xx_2, "data/xx_2")
save(xx_11, "data/xx_11")
save(xx_all, "data/xx_all")


In [92]:
print("The weight 13 Euler characteristic in power sum basis, and yet transposed")
%time wt13euler_xp = xx_all - xx_2*xx_11/u

print("to Schur basis and transpose ...")
%time wt13euler=pstosu(transpose_yng(wt13euler_xp))

print("display ...")
DisplayPoly(wt13euler)

save(wt13euler, "data/lastwt13euler")

The weight 13 Euler characteristic in power sum basis, and yet transposed
CPU times: user 1.56 s, sys: 75.7 ms, total: 1.64 s
Wall time: 1.65 s
to Schur basis and transpose ...
CPU times: user 2.92 s, sys: 15.1 ms, total: 2.93 s
Wall time: 2.94 s
display ...
g= 1 , n= 12 :  -s[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
g= 1 , n= 13 :  s[3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] + s[3, 2, 1, 1, 1, 1, 1, 1, 1, 1] + s[4, 1, 1, 1, 1, 1, 1, 1, 1, 1]
g= 1 , n= 14 :  -s[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] - s[2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1] - 2*s[3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1] - s[3, 2, 2, 1, 1, 1, 1, 1, 1, 1] - 2*s[3, 3, 1, 1, 1, 1, 1, 1, 1, 1] - s[3, 3, 2, 1, 1, 1, 1, 1, 1] - 2*s[4, 2, 1, 1, 1, 1, 1, 1, 1, 1] - s[4, 3, 1, 1, 1, 1, 1, 1, 1] - s[5, 1, 1, 1, 1, 1, 1, 1, 1, 1] - s[5, 2, 1, 1, 1, 1, 1, 1, 1] - s[6, 1, 1, 1, 1, 1, 1, 1, 1]
g= 1 , n= 15 :  s[2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1] + s[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1] + s[3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] + 2*s[3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [99]:
# Create Latex Table (optional)
toLatexTable(wt13euler,13, 13, 14)

\begin{tabular}{|g|M|M|M|M|M|M|M|M|M|M|M|M|M|M|} \hline \rowcolor{Gray} g,n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13\\ 
\hline
0 & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ \\  
\hline
 1 & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ -s_{2,1,1,1,1,1,1,1,1,1,1} $ & $ s_{3,1,1,1,1,1,1,1,1,1,1} + s_{3,2,1,1,1,1,1,1,1,1} + s_{4,1,1,1,1,1,1,1,1,1} $ \\  
\hline
 2 & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ s_{1,1,1,1,1,1,1,1,1,1} $ & $ s_{1,1,1,1,1,1,1,1,1,1,1} - s_{2,1,1,1,1,1,1,1,1,1} - s_{2,2,1,1,1,1,1,1,1} - 2 s_{3,1,1,1,1,1,1,1,1} $ & $ s_{2,2,1,1,1,1,1,1,1,1} + 3 s_{3,2,1,1,1,1,1,1,1} + s_{3,2,2,1,1,1,1,1} + 2 s_{3,3,1,1,1,1,1,1} + 3 s_{4,1,1,1,1,1,1,1,1} + 2 s_{4,2,1,1,1,1,1,1} + 2 s_{5,1,1,1,1,1,1,1} $ & $ $ \\  
\hline
 3 & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ 0 $ & $ s_{1,1